In [1]:
import click
import numpy as np
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
import pandas as pd
from src.data.make_dataset import make_dataset
from src.features.make_features import make_features
from src.model.main import make_model
import joblib
import pickle
import gzip
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from flair.data import Sentence
from flair.models import SequenceTagger

2023-11-05 09:25:58,859 SequenceTagger predicts: Dictionary with 69 tags: <unk>, O, DET, NFP, ADJFP, AUX, VPPMS, ADV, PREP, PDEMMS, NMS, COSUB, PINDMS, PPOBJMS, VERB, DETFS, NFS, YPFOR, VPPFS, PUNCT, DETMS, PROPN, ADJMS, PPER3FS, ADJFS, COCO, NMP, PREL, PPER1S, ADJMP, VPPMP, DINTMS, PPER3MS, PPER3MP, PREF, ADJ, DINTFS, CHIF, XFAMIL, PRELFS, SYM, NOUN, MOTINC, PINDFS, PPOBJMP, NUM, PREFP, PDEMFS, VPPFP, PPER3FP


In [18]:
tagger = SequenceTagger.load("qanastek/pos-french")
input_filename=("../data/raw/train.csv")

df_base = make_dataset(input_filename)
df_base

2023-11-05 12:33:37,765 SequenceTagger predicts: Dictionary with 69 tags: <unk>, O, DET, NFP, ADJFP, AUX, VPPMS, ADV, PREP, PDEMMS, NMS, COSUB, PINDMS, PPOBJMS, VERB, DETFS, NFS, YPFOR, VPPFS, PUNCT, DETMS, PROPN, ADJMS, PPER3FS, ADJFS, COCO, NMP, PREL, PPER1S, ADJMP, VPPMP, DINTMS, PPER3MS, PPER3MP, PREF, ADJ, DINTFS, CHIF, XFAMIL, PRELFS, SYM, NOUN, MOTINC, PINDFS, PPOBJMP, NUM, PREFP, PDEMFS, VPPFP, PPER3FP


,Unnamed: 0,video_name,is_name,is_comic,comic_name,tokens
0,0,Le Barbecue Disney - La chanson de Frédéric Fr...,"[0, 0, 0, 0, 0, 0, 0, 1, 1]",1,"[""Frédéric Fromet""]","[""Le"", ""Barbecue"", ""Disney"", ""-"", ""La"", ""chans..."
1,1,Le Roi et l'Oiseau - La Chronique de Christine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",1,"[""Christine Gonzalez""]","[""Le"", ""Roi"", ""et"", ""l'"", ""Oiseau"", ""-"", ""La"",..."
2,2,L'amour du lac - La chronique d'Hippolyte Gira...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",1,"[""Hippolyte Girardot""]","[""L'"", ""amour"", ""du"", ""lac"", ""-"", ""La"", ""chron..."
3,3,La fille de la piscine de Léa Tourret - La chr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",0,[],"[""La"", ""fille"", ""de"", ""la"", ""piscine"", ""de"", ""..."
4,4,"""Le soleil va moins faire son malin quand Jean...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0,[],"[""\""Le"", ""soleil"", ""va"", ""moins"", ""faire"", ""so..."
...,...,...,...,...,...,...
994,994,Au Ces de Las Vegas des innovations à gogo et ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,[],"[""Au"", ""Ces"", ""de"", ""Las"", ""Vegas"", ""des"", ""in..."
995,995,Le questionnaire Jupiproust de Frédérique Matonti,"[0, 0, 0, 0, 0, 0]",1,[],"[""Le"", ""questionnaire"", ""Jupiproust"", ""de"", ""F..."
996,996,Raphael - La Nuit N'en Finit Plus (cover),"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,[],"[""Raphael"", ""-"", ""La"", ""Nuit"", ""N'"", ""en"", ""Fi..."
997,997,Des sachets de drogue estampillés Haribeu sais...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,[],"[""Des"", ""sachets"", ""de"", ""drogue"", ""estampill\..."


In [3]:
df_base["is_name"] = df_base["is_name"].apply(eval)
df_base["tokens"] = df_base["tokens"].apply(eval)
df_base

,Unnamed: 0,video_name,is_name,is_comic,comic_name,tokens
0,0,Le Barbecue Disney - La chanson de Frédéric Fr...,"[0, 0, 0, 0, 0, 0, 0, 1, 1]",1,"[""Frédéric Fromet""]","[Le, Barbecue, Disney, -, La, chanson, de, Fré..."
1,1,Le Roi et l'Oiseau - La Chronique de Christine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",1,"[""Christine Gonzalez""]","[Le, Roi, et, l', Oiseau, -, La, Chronique, de..."
2,2,L'amour du lac - La chronique d'Hippolyte Gira...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",1,"[""Hippolyte Girardot""]","[L', amour, du, lac, -, La, chronique, d', Hip..."
3,3,La fille de la piscine de Léa Tourret - La chr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]",0,[],"[La, fille, de, la, piscine, de, Léa, Tourret,..."
4,4,"""Le soleil va moins faire son malin quand Jean...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0,[],"[""Le, soleil, va, moins, faire, son, malin, qu..."
...,...,...,...,...,...,...
994,994,Au Ces de Las Vegas des innovations à gogo et ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,[],"[Au, Ces, de, Las, Vegas, des, innovations, à,..."
995,995,Le questionnaire Jupiproust de Frédérique Matonti,"[0, 0, 0, 0, 0, 0]",1,[],"[Le, questionnaire, Jupiproust, de, Frédérique..."
996,996,Raphael - La Nuit N'en Finit Plus (cover),"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,[],"[Raphael, -, La, Nuit, N', en, Finit, Plus, (c..."
997,997,Des sachets de drogue estampillés Haribeu sais...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,[],"[Des, sachets, de, drogue, estampillés, Haribe..."


In [4]:
def pos_tagging(phrase, is_name):
    result = pd.DataFrame(columns=['words', 'tags'])
    for i in range(len(phrase)):
        word = phrase[i]

        if(word):
            sentence = Sentence(word)
            tagger.predict(sentence)
        
            # First word : <s> (starting)
            word = word + "<s>" if i == 0 else word
            # Last word : <e> (end)
            word = word + "<e>" if i == (len(phrase) - 1) else word
            # Capitalized word : <c> (caps)
            word = word + "<c>" if len(word.replace('"', '').strip()) > 0 and (word.replace('"', '').strip())[0].isupper() else word
        
            values = pd.DataFrame([{'words': word, 'tags': sentence.tag}])
            
            #else:
            #    values = pd.DataFrame([{'words': word, 'tags': ''}])
                
            result = pd.concat([result, values], axis=0, ignore_index=True)
        
    result['is_name'] = pd.DataFrame({'is_name': is_name})
    
    return(result)

In [5]:
df = pd.DataFrame(columns=['words', 'tags', 'is_name'])

for i in df_base.index:
    tokens = df_base.loc[i, 'tokens']
    names = [int(i) for i in df_base.loc[i, 'is_name']]
    result = pos_tagging(tokens, names)
    df = pd.concat([df, result], axis=0, ignore_index=True)
    
df

Word:Le.
Word:Barbecue.
Word:Disney.
Word:-.
Word:La.
Word:chanson.
Word:de.
Word:Frédéric.
Word:Fromet.
Word:Le.
Word:Roi.
Word:et.
Word:l'.
Word:Oiseau.
Word:-.
Word:La.
Word:Chronique.
Word:de.
Word:Christine.
Word:Gonzalez.
Word:L'.
Word:amour.
Word:du.
Word:lac.
Word:-.
Word:La.
Word:chronique.
Word:d'.
Word:Hippolyte.
Word:Girardot.
Word:La.
Word:fille.
Word:de.
Word:la.
Word:piscine.
Word:de.
Word:Léa.
Word:Tourret.
Word:-.
Word:La.
Word:chronique.
Word:de.
Word:Juliette.
Word:Arnaud.
Word:"Le.
Word:soleil.
Word:va.
Word:moins.
Word:faire.
Word:son.
Word:malin.
Word:quand.
Word:Jean-Luc.
Word:va.
Word:aller.
Word:lui.
Word:hurler.
Word:dessus.
Word:"les.
Word:températures.
Word:c'.
Word:e.
Word:Seins.
Word:en.
Word:quête.
Word:d’une.
Word:libération.
Word:-.
Word:La.
Word:chronique.
Word:d'.
Word:Isabelle.
Word:Sorente.
Word:Pickles.
Word:de.
Word:navet.
Word:express.
Word:-.
Word:Les.
Word:recettes.
Word:de.
Word:François-Régis.
Word:Gaudry.
Word:La.
Word:Belle.
Word:et.
Word:l

,words,tags,is_name
0,Le<s><c>,DETMS,0
1,Barbecue<c>,NMS,0
2,Disney<c>,PROPN,0
3,-,PUNCT,0
4,La<c>,DETFS,0
...,...,...,...
12285,Bd<c>,PROPN,0
12286,Fnac<c>,PROPN,0
12287,France<c>,PROPN,0
12288,Inter<c>,PROPN,0


In [6]:
df.groupby('tags').size().reset_index(name='counts')

,tags,counts
0,ADJ,31
1,ADJFP,56
2,ADJFS,297
3,ADJMP,96
4,ADJMS,137
5,ADV,370
6,AUX,142
7,CHIF,114
8,COCO,202
9,COSUB,68


In [7]:
X = df.drop('is_name', axis=1)
y = df.is_name.values

X

,words,tags
0,Le<s><c>,DETMS
1,Barbecue<c>,NMS
2,Disney<c>,PROPN
3,-,PUNCT
4,La<c>,DETFS
...,...,...
12285,Bd<c>,PROPN
12286,Fnac<c>,PROPN
12287,France<c>,PROPN
12288,Inter<c>,PROPN


In [8]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [9]:
X.shape, y.shape

((12290, 2), (12290,))

In [13]:
pipeline = Pipeline([
            ("vectorizer", DictVectorizer(sparse=False)),
            ("classifier", GradientBoostingClassifier(n_estimators=150, criterion='squared_error')),
        ])

In [14]:
results = cross_validate(pipeline, X.to_dict('records'), y.astype(int), scoring="accuracy", cv=10, return_estimator=True)

In [15]:
scores = results["test_score"]

print(f"Accuracy per fold : {np.around(scores, 2)}")
print(f"Global accuracy {100 * round(np.mean(scores), 2)}% + {round(np.std(scores), 2)}")

Accuracy per fold : [0.96 0.97 0.96 0.96 0.96 0.96 0.96 0.96 0.96 0.96]
Global accuracy 96.0% + 0.0
